# 카카오톡 대화 데이터셋

In [ ]:
import pandas as pd
import unicodedata as ud
import re
from pathlib import Path
from sklearn.model_selection import train_test_split

# --- 기본 설정 ---
INPUT_PATH = "/content/KakaoData.csv"
OUT_DIR = Path("/content/kakao_clean_final3")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# --- 안전하게 CSV 읽기 ---
def safe_read_csv(path):
    for enc in ["utf-8", "utf-8-sig", "cp949", "euc-kr"]:
        try:
            return pd.read_csv(path, encoding=enc)
        except Exception:
            pass
    raise ValueError("CSV 인코딩 문제 발생")

df = safe_read_csv(INPUT_PATH)
df.columns = [c.strip() for c in df.columns]
df = df[["req", "res"]].dropna()

# --- 정규식 패턴 정의 ---
# 한글 자모 (ㅋ,ㅎ,ㅠ,ㅜ 등) + 분해 자모 (ᄏ, ᅲ 등)
EMOTE_CLASS = r"[\u314B\u314E\u3161\u315C\u110F\u1112\u1172\u116E]"
# 기호류: ^^, ㅡㅡ, :, ;, ~, -, _, = 등
SYMBOL_CLASS = r"[\^;:~\-_=]+|[ㅡ]+"
# 의미 없는 감탄사 (문장 맨앞 또는 전체 단어로 존재)
FILLER_WORDS = [
    # 단순 감탄사
    "음", "응", "아", "어", "오", "흠", "앗", "웅", "아하", "오호", "에휴", "엥", "그냥", "아니", "힉", "휴", "헤헷", "야쓰", "그",
    "와", "헉"
    # 물음표 붙은 감탄사
    "음?", "응?", "아?", "어?", "오?", "흠?", "웅?", "앗?", "아하?", "오호?", "엥?", "엥...?",
    # 점(.) 붙은 감탄사 (아.., 음..., 응.... 등)
    "음.", "응.", "아.", "어.", "오.", "흠.", "웅.", "앗.", "아하.", "오호.", "엥.",
    "음..", "응..", "아..", "어..", "오..", "흠..", "웅..", "앗..", "아하..", "오호..", "엥..",
    "음...", "응...", "아...", "어...", "오...", "흠...", "웅...", "앗...", "아하...", "오호...", "엥...", "와...",
]

# 감탄사 패턴 (문장 앞뒤 공백 포함 또는 문장 전체일 때)
# 리스트를 정규식으로 변환 (단어 경계 기준)
FILLER_RE = re.compile(
    rf"(^|\s)({'|'.join(map(re.escape, FILLER_WORDS))})(\s|$)"
)

# 미리 컴파일한 정규식
RE_EMOTE = re.compile(rf"(?:{EMOTE_CLASS}|ㅋ|ㅎ|ㅠ|ㅜ)+")
RE_SYMBOL = re.compile(rf"(?:{SYMBOL_CLASS})+")
RE_URL = re.compile(r"https?://\S+")
RE_EMAIL = re.compile(r"\S+@\S+\.\S+")

# --- 정규화 및 전처리 함수 ---
def normalize_and_clean(s: str) -> str:
    if not isinstance(s, str):
        return ""
    # 1️⃣ 유니코드 정규화 및 제로폭 문자 제거
    s = ud.normalize("NFKC", s)
    s = re.sub(r"[\u200B-\u200D\uFEFF]", "", s)
    s = "".join(ch for ch in s if ch.isprintable())
    s = s.strip()

    # 2️⃣ 감정/기호 제거
    s = RE_EMOTE.sub("", s)     # ㅋㅋ, ㅎㅎ, ㅠㅠ, ᄏᄏ, ᅲ 등
    s = RE_SYMBOL.sub("", s)    # ^^, ㅡㅡ, :, ; 등
    # 3️⃣ 의미 없는 감탄사 제거
    s = FILLER_RE.sub(" ", s)
    # 4️⃣ URL/이메일 제거
    s = RE_URL.sub("", s)
    s = RE_EMAIL.sub("", s)
    # 5️⃣ 공백 정리
    s = re.sub(r"\s+", " ", s).strip()
    return s

# --- 전처리 적용 ---
df["req_clean"] = df["req"].apply(normalize_and_clean)
df["res_clean"] = df["res"].apply(normalize_and_clean)

# --- 짧거나 긴 문장 제거 ---
df = df[df["req_clean"].str.len() > 1]
df = df[df["res_clean"].str.len() > 1]
df = df[df["req_clean"].str.len() < 300]
df = df[df["res_clean"].str.len() < 300]

# --- 중복/에코 제거 ---
df = df[df["req_clean"] != df["res_clean"]]
df = df.drop_duplicates(subset=["req_clean", "res_clean"]).reset_index(drop=True)

# --- 결과 저장 ---
clean_csv = OUT_DIR / "KakaoData_clean_final3.csv"
df[["req_clean", "res_clean"]].rename(columns={"req_clean":"req","res_clean":"res"}).to_csv(
    clean_csv, index=False, encoding="utf-8-sig"
)

print(f"✅ 전처리 완료! 감정·기호·감탄사 제거 후 {len(df)}개 대화쌍 저장됨")
print(f"📂 저장 경로: {clean_csv}")

In [ ]:
df4 = pd.read_csv('/content/kakao_clean_final3/KakaoData_clean_final3.csv')
df4

In [ ]:
import pandas as pd
import json
import re
import unicodedata as ud
from pathlib import Path

# ===== 설정 =====
INPUT_CSV  = "/content/cafe_qa_korean_polite_v2.csv"
OUT_JSONL  = "/content/cafe_qa_korean_polite_v2.jsonl"

DEFAULT_CONTEXT = "카페"
DEFAULT_SPEAKER = "clerk"

REQ_COL = "req"
RES_COL = "res"

def safe_read_csv(path):
    for enc in ["utf-8", "utf-8-sig", "cp949", "euc-kr"]:
        try:
            return pd.read_csv(path, encoding=enc)
        except Exception:
            pass
    raise ValueError("CSV 인코딩을 확인해주세요.")

# ===== 유틸: 간단 정규화 & 공백 정리 =====
def norm_text(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = ud.normalize("NFKC", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# ===== 토큰 분할 정책 =====
def tokenize_korean_sentence(s: str):
    s = norm_text(s)
    if not s:
        return []
    # 공백 기준 토큰화
    tokens = s.split(" ")
    # 빈 토큰 제거
    tokens = [t for t in tokens if t]
    return tokens

# ===== 메인 변환 =====
df = safe_read_csv(INPUT_CSV)
# 컬럼 이름 공백 제거
df.columns = [c.strip() for c in df.columns]

# 필수 컬럼 존재 확인
assert REQ_COL in df.columns and RES_COL in df.columns, f"CSV에 '{REQ_COL}', '{RES_COL}' 컬럼이 있는지 확인해주세요. 현재: {df.columns}"

n_rows = 0
n_lines = 0

with open(OUT_JSONL, "w", encoding="utf-8") as fout:
    for _, row in df.iterrows():
        user_asr = norm_text(str(row[REQ_COL]))
        target   = norm_text(str(row[RES_COL]))

        # 목표 문장 토큰화
        tokens = tokenize_korean_sentence(target)
        if len(tokens) == 0 or len(user_asr) == 0:
            continue

        # 규칙
        for i, tok in enumerate(tokens):
            rec = {
                "context": DEFAULT_CONTEXT,
                "speaker": DEFAULT_SPEAKER,
                "user_asr": user_asr,
                "selected_chunks": tokens[:i],
                "target_chunk": tok
            }
            fout.write(json.dumps(rec, ensure_ascii=False) + "\n")
            n_lines += 1

        n_rows += 1

print(f"✅ 변환 완료! 총 {n_rows}개 문장에서 {n_lines}개 라인 생성")
print(f"📄 출력 파일: {OUT_JSONL}")


# 카페 추가 데이터셋

In [ ]:
import pandas as pd
import json
import re
import unicodedata as ud
from pathlib import Path

# ===== 설정 =====
INPUT_CSV  = "/content/cafe_qa_korean_polite_v2.csv"   # 업로드한 CSV 경로
OUT_JSONL  = "/content/cafe_qa_korean_polite_v2.jsonl"           # 출력 JSONL 경로

DEFAULT_CONTEXT = ""          # 예: "카페" 같이 넣고 싶으면 여기 적기
DEFAULT_SPEAKER = "clerk"     # 예: "clerk", "agent", "assistant" 등

# CSV의 컬럼명이 다르면 여기서 바꿔줘:
REQ_COL = "req"               # 사용자 발화(질문/요청) → user_asr로 들어감
RES_COL = "res"               # 모델/점원이 말할 목표 문장 → 조각으로 분리할 대상

# ===== 유틸: 안전한 CSV 읽기 =====
def safe_read_csv(path):
    for enc in ["utf-8", "utf-8-sig", "cp949", "euc-kr"]:
        try:
            return pd.read_csv(path, encoding=enc)
        except Exception:
            pass
    raise ValueError("CSV 인코딩을 확인해주세요.")

# ===== 유틸: 간단 정규화 & 공백 정리 (선택) =====
def norm_text(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = ud.normalize("NFKC", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# ===== 토큰 분할 정책 =====
# 기본은 공백 기반. 이미 전처리에서 이모/기호/중복 공백을 정리했으니 일반적으로 충분함.
# 필요하면 아래에서 한국어 토크나이저(MeCab 등)로 교체 가능.
def tokenize_korean_sentence(s: str):
    s = norm_text(s)
    if not s:
        return []
    # 공백 기준 토큰화
    tokens = s.split(" ")
    # 빈 토큰 제거
    tokens = [t for t in tokens if t]
    return tokens

# ===== 메인 변환 =====
df = safe_read_csv(INPUT_CSV)
# 컬럼 이름 공백 제거
df.columns = [c.strip() for c in df.columns]

# 필수 컬럼 존재 확인
assert REQ_COL in df.columns and RES_COL in df.columns, f"CSV에 '{REQ_COL}', '{RES_COL}' 컬럼이 있는지 확인해주세요. 현재: {df.columns}"

n_rows = 0
n_lines = 0

with open(OUT_JSONL, "w", encoding="utf-8") as fout:
    for _, row in df.iterrows():
        user_asr = norm_text(str(row[REQ_COL]))
        target   = norm_text(str(row[RES_COL]))

        # 목표 문장 토큰화
        tokens = tokenize_korean_sentence(target)
        if len(tokens) == 0 or len(user_asr) == 0:
            continue

        # 예시와 동일한 규칙:
        #  ["아이스","아메리카노","한","잔","주세요"] -> 5줄 생성
        #   selected_chunks=[],        target_chunk="아이스"
        #   selected_chunks=["아이스"], target_chunk="아메리카노"
        #   ...
        for i, tok in enumerate(tokens):
            rec = {
                "context": DEFAULT_CONTEXT,
                "speaker": DEFAULT_SPEAKER,
                "user_asr": user_asr,
                "selected_chunks": tokens[:i],
                "target_chunk": tok
            }
            fout.write(json.dumps(rec, ensure_ascii=False) + "\n")
            n_lines += 1

        n_rows += 1

print(f"✅ 변환 완료! 총 {n_rows}개 문장에서 {n_lines}개 라인 생성")
print(f"📄 출력 파일: {OUT_JSONL}")